

This file includes the work that is distributed in the Apache License 2.0.

このファイルには、 Apache 2.0ライセンスで配布されている製作物が含まれています。

I modified the file below to create this file.

このファイルは、以下の制作物の一部を変更して再利用しています。

https://github.com/google/earthengine-community/blob/master/tutorials/sentinel-2-s2cloudless/index.ipynb


In [3]:
#@title Copyright 2020 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
#@title Copyright 2022 Yoshihiro Sakatani { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Sentinel-2 観測データを使った火災発生の推定




## 準備

In [1]:
import ee
import folium

#### GEEの認証と初期化

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=PvWxOpR7APRbHoiCxd6GsP3HIw75MByA3SfUfYTf1kI&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiCwi2TZLfnzN-phT4ZFEDCtmVcnG8cLLYa5MgKfDOTSFOq3tNYYAU

Successfully saved authorization token.


#### パラメータの設定

取得データ関連

In [42]:
# Sentinel-2, Level-2A
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR
DATA_NAME = 'COPERNICUS/S2_SR'

# 関心域の座標（イルピン）
AOI = ee.Geometry.Point(30.2, 50.55)

# 日付
START_DATE = '2022-03-23'
END_DATE = '2022-03-24'

# 被雲率の閾値
TH_CLOUD = 60

火災推定関連

In [43]:
# バンド名
SWIR2 = 'B12'
NIR = 'B8A'
RED = 'B4'

# SWIR2の閾値
TH_SWIR2 = 0.5

# SWIR2-Red比の閾値
TH_SWIR2_RED_RATIO = 2.0

# NBRの閾値
TH_NBR = 0.3

地図表示関連

In [44]:
# 拡大率の初期値
ZOOM_START = 13

## 観測データの取得
`ee.ImageCollection`でイメージを取得します。引数には、データの名前を指定します。

`filterBounds`、`filterDate`を使って、領域や日付でデータをフィルタリングできます。

また、`filter`を使って、任意のプロパティでフィルタリングできます。

以下は、被雲率でのフィルタリング例です。

In [45]:
def get_img_col(data_name, aoi, start_date, end_date, th_cloud=60):
    return (ee.ImageCollection(data_name)
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', th_cloud)))


In [46]:
img_col = get_img_col(DATA_NAME, AOI, START_DATE, END_DATE, TH_CLOUD)

データの個数の確認。

以下が0になっていると、設定した領域、期間、被雲率の組み合わせを満たすデータが無いので、どちらかを変更します。

In [47]:
img_col.size().getInfo()

2

## 観測データの加工

#### 火災レイヤーの作成

参考文献

NBR
- https://www.fs.fed.us/rm/pubs_journals/2018/rmrs_2018_cohen_w001.pdf
- https://www.fs.fed.us/rm/pubs/rmrs_gtr164/rmrs_gtr164_13_land_assess.pdf

AFD-S2
- https://www.sciencedirect.com/science/article/pii/S0303243421000544


In [48]:
def add_fire_inference(img):

    # NBRの計算
    nbr = img.normalizedDifference([SWIR2, NIR]).rename('nbr')
    nbr_bool = nbr.gt(TH_NBR).rename('nbr_bool')

    # AFD-S2をヒントにした二直線分離（rはroughの意です）
    swir2_red_ratio_exp = f'(b("{SWIR2}") / b("{RED}"))'
    swir2_red_ratio = img.expression(swir2_red_ratio_exp)
    swir2_red_ratio_bool = swir2_red_ratio.gt(TH_SWIR2_RED_RATIO)

    swir2_bool = img.select(SWIR2).gt(TH_SWIR2)

    r_afd_s2_c1 = swir2_red_ratio_bool.multiply(swir2_bool)

    r_afd_s2_swir2 = img.select([SWIR2]).multiply(r_afd_s2_c1).rename('r_afd_s2_swir2')
    r_afd_s2_nir = img.select([NIR]).multiply(r_afd_s2_c1).rename('r_afd_s2_nir')
    r_afd_s2_red = img.select([RED]).multiply(r_afd_s2_c1).rename('r_afd_s2_red')

    return img.addBands(ee.Image([nbr, nbr_bool, swir2_red_ratio, r_afd_s2_swir2, r_afd_s2_nir, r_afd_s2_red]))

In [49]:
s2_sr_fire_col = img_col.map(add_fire_inference)

## 地図への表示

`folium.raster_layers.TileLayer`をラップして、foliumで表示する地図にレイヤーを追加する`folium.Map.add_ee_layer`を以下のように定義します。

In [50]:
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        overlay=True,
        control=True
        ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

地図を作成して、その上にレイヤーを重ねて表示します。

In [54]:
def make_layers(col):
    # イメージコレクション内の全画像の合成
    #（同じ地点の異なる日時の画像が含まれる場合は、新しいものが優先される）
    img = col.mosaic()

    nbr = img.select('nbr')
    nbr_bool = img.select('nbr_bool').selfMask()

    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=ZOOM_START)

    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', False)
    m.add_ee_layer(img,
                   {'bands': ['TCI_R', 'TCI_G', 'TCI_B']},
                   'TCI')
    m.add_ee_layer(img,
                   {'bands': ['B12', 'B8A', 'B4'], 'min': 0, 'max': 10000},
                   'SWIR', False)
    m.add_ee_layer(nbr, {'min': 0, 'max': 1},
                   'NBR', False, 0.7)
    m.add_ee_layer(nbr_bool,
                   {'palette': 'FF3616'},
                   'NBR_bool', True, 0.6)
    m.add_ee_layer(img,
                   {'bands': ['r_afd_s2_swir2', 'r_afd_s2_nir', 'r_afd_s2_red'], 'min': 0, 'max': 10000},
                   'rAFD-S2', True, 0.6)

    m.add_child(folium.LayerControl())

    return m

In [55]:
m = make_layers(s2_sr_fire_col)
m

右上のレイヤーアイコンから、表示するレイヤーを選択できます。

# 保存
可視化の結果をhtmlに保存したい場合は、GoogleDriveをマウントして保存してください。

In [64]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [65]:
import os
os.chdir('/content/drive/My Drive/research/sentinel/')

In [66]:
point = AOI.coordinates().reverse().getInfo()

In [67]:
m.save(f"{DATA_NAME.replace('/', '-')}_{START_DATE}_{END_DATE}_{point[0]}_{point[1]}_{TH_CLOUD}.html")

以上